In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
dg_file = "Resources/disc_golf_courses.csv"
ff_file = "Resources/Fast_Food_Restaurants_US.csv"

## READ CSV FILES

#ff_df = pd.read_csv(ff_file)
dg_df = pd.read_csv(dg_file)
ff_df = pd.read_csv(ff_file)

#delete unwanted columns
del ff_df['Unnamed: 0']
del dg_df['id']
del dg_df['holeCount']
del dg_df['rating']
del ff_df['latitude']
del ff_df['longitude']
del dg_df['latitude']
del dg_df['longitude']
del ff_df['websites']
del dg_df['zip']
del ff_df['country']
del ff_df['postalCode']

In [3]:
ff_df = ff_df.drop_duplicates(subset=['address'])

In [4]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

ff_df['province'] = ff_df['province'].str.strip().replace(states)
ff_df = ff_df.rename(columns = {'province':'state'})

In [5]:
ff_df = ff_df.drop(['address'], axis=1)
ff_df['categories'] = 'Fast Food Restaurant'

In [6]:
ff_df = ff_df[['name', 'city', 'state', 'categories']]

In [7]:
dg_df['categories'] = 'Disc Golf Course'

In [8]:
concat_df = pd.concat([ff_df, dg_df], ignore_index=True)

In [10]:
concat_df.sort_values(by=['state'])

,name,city,state,categories
9345,Golden Bear DGC,Lexington,Alabama,Disc Golf Course
9396,Sparkman Park - Lakeside,Hartselle,Alabama,Disc Golf Course
9395,Southside Park,Huntsville,Alabama,Disc Golf Course
9394,Smiths Station Disc Golf Course,Smiths Station,Alabama,Disc Golf Course
9393,Shocco Springs,Talladega,Alabama,Disc Golf Course
...,...,...,...,...
16216,Lakeside Lodge,Pinedale,Wyoming,Disc Golf Course
16215,La Grange Disc Golf Course,La Grange,Wyoming,Disc Golf Course
16214,Jackson Hole Mountain Resort Disc Golf Course,Jackson Hole,Wyoming,Disc Golf Course
2729,Arby's,Rock Springs,Wyoming,Fast Food Restaurant


In [11]:
concat_df.describe()

,name,city,state,categories
count,16310,16310,16310,16310
unique,7277,6612,50,2
top,McDonald's,Houston,California,Fast Food Restaurant
freq,1727,99,1444,9302


In [12]:
concat_df.groupby(['state', 'categories']).size().sort_values(ascending=False).unstack()

categories,Disc Golf Course,Fast Food Restaurant
state,,
Alabama,122,6
Alaska,38,14
Arizona,77,295
Arkansas,116,90
California,333,1111
Colorado,204,135
Connecticut,31,51
Delaware,13,38
Florida,182,574


In [13]:
concat_df.groupby('city').size().sort_values(ascending=False)

city
Houston             99
Las Vegas           78
Columbus            67
Phoenix             66
Los Angeles         60
                    ..
Indian River         1
Incline Village      1
Ina                  1
Imperial Beach       1
Zumbrota             1
Length: 6612, dtype: int64

In [14]:
rds_connection_string = "postgres:postgres@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [15]:
engine.table_names()

<ipython-input-15-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['golf_eat']

In [16]:
concat_df.to_sql(name='golf_eat_concat', con=engine, if_exists='append', index=False)